In [1]:
# Modules for bead image analysis

In [2]:
import numpy as np
from PIL import Image
from matplotlib import pyplot as plt
%matplotlib inline
#from tqdm import tqdm
import os
import sys
import cv2
import csv
import nd2reader as nd2; print("nd2 reader version: {0}".format(nd2.__version__))
from statistics import mean, median, stdev
import pandas as pd
#warnings.simplefilter("ignore", RuntimeWarning)

nd2 reader version: 3.2.3


/home2/software_projects/beadAnalysis/env/venv_beads/lib/python3.8/site-packages/pims/image_reader.py:26: RuntimeWarning: PIMS image_reader.py could not find scikit-image. Falling back to matplotlib's imread(), which uses floats instead of integers. This may break your scripts. 
(To ignore this warning, include the line "warnings.simplefilter("ignore", RuntimeWarning)" in your script.)
  warnings.warn(RuntimeWarning(ski_preferred))


In [3]:
class ImageStack(object):
    """Analysis of beads (from individual nd2 files) to extract all relevant information
        NOTE: The file form of the nd2 file must be only channels and no frames/time/zstacks etc; 
        TODO: A different nd2 stack will mess up the processing and analysis and will need reprocessiong
    """
    
    def __init__(self,bead_nd2_fpath,mono_channel=0):
        self.bead_nd2_fpath = bead_nd2_fpath
        self.metadata, self.nd2image = self.metadata()
        self.check_nd2_file_form()
      
    def check_nd2_file_form(self):
        assert (len(self.metadata['fields_of_view']) == 1 and len(self.metadata['frames']) == 1),"Metadata {0}".format(self.metadata)
        assert ((len(self.metadata['channels']))>1), "Not enough channels in this image stack"
        return True
    
    def metadata(self):
        nd2image = nd2.ND2Reader(self.bead_nd2_fpath)
        metadata = nd2image.metadata
        return metadata, nd2image
    
    def get_mono_image(self):
        return self.nd2image.get_frame(0)
    
    def channels(self):
        return self.metadata['channels']
    
    def frames(self):
        return self.metadata['frames']
   
    def fname(self):
        return(os.path.split(self.bead_nd2_fpath)[1])


In [4]:
test_tentagel_folder = "/home2/software_projects/beadAnalysis/temp/test_images/2020_10_15/Tentagel_DyeScreen_expt/Atto495"
test_nd2_fname = '201015_TGNH2_Atto495_MEOH_10X_500ms_flds001.nd2'
test_nd2_fpath = os.path.join(test_tentagel_folder,test_nd2_fname)
t = ImageStack(test_nd2_fpath)

/home2/software_projects/beadAnalysis/env/venv_beads/lib/python3.8/site-packages/nd2reader/raw_metadata.py:171: UserWarning: Z-levels details missing in metadata. Using Z-coordinates instead.
  warnings.warn("Z-levels details missing in metadata. Using Z-coordinates instead.")


In [5]:
class Beads(ImageStack):
    """
    The Beads class actually tracks the intensity of the beads across the different channels. 
    TODO: Ensures that it is done across channels and the ImageStack inherited is of the form
    Summarizing a dataframe to obtain values
    """
    def __init__(self,bead_nd2_file,mono_channel=0):
        super().__init__(bead_nd2_file)
        self.col_headers = self.make_column_header()
        self.mono_channel = mono_channel
    
    def make_column_header(self):
        channels = self.channels()
        channel_dependent_headers = list()
        col_headers = ["Filename", "Frame ID", "Mask ID", "Mask-X0","Mask-Y0","oRadius (pix)",
                       "Circular Area (pix^2)", "Ring Area (pix^2)", "inner Circle Area (pix^2)"]
        
        for channel_name in channels:
            _header_list = [channel_name + '---' + _head_ 
                            for _head_ in ('oCircle Intensity', 'oCircle Mean Intensity', 'oCircle Median Intensity',
                                           'iCircle Intensity', 'iCircle Mean Intensity', 'iCircle Median Intensity',
                                           'ring Intensity', 'ring Mean Intensity', 'ring Median Intensity')]
            channel_dependent_headers.extend(_header_list)
        col_headers.extend(channel_dependent_headers)
        
        return col_headers
    
    def make_dataframe(self,row_values):
        self.allbeads_df = pd.DataFrame(row_values,columns=self.col_headers)
        display(self.allbeads_df)
        return True
    
    def preprocess_image(self,img,smoothing='median'):
        mod_img1 = img.astype(np.float32)
        mod_img2 = (mod_img1/256).astype('uint8') #Need an 8-bit, single-channel, grayscale input
        # Blurring
        if smoothing == 'median': cv2.medianBlur(mod_img2,5)
        if smoothing == 'gaussian': cv2.GaussianBlue(mod_img2,(5,5))
        return mod_img2

    def filter_beads(self,circles, min_width=100,max_width=170):
        all_circles = list()
        for circle in circles:
            r = int(circle[2])
            if r in range(min_width,max_width):
                all_circles.append(circle)
        return all_circles

    def find_beads(self,*args, **kwargs):
        """
        hough_params = dict(dp=1.5,minDist=250,param1=45,param2=45,minRadius=100,maxRadius=700)
        circles = t.find_beads(mono_image,**hough_params)
        """
        processed_img = self.preprocess_image(self.get_mono_image())
        circles = (cv2.HoughCircles(processed_img,cv2.HOUGH_GRADIENT,
                                    dp = kwargs['dp'],
                                    minDist = kwargs['minDist'],
                                    param1 = kwargs['param1'],
                                    param2 = kwargs['param2'],
                                    minRadius = kwargs['minRadius'],
                                    maxRadius = kwargs['maxRadius']))
        circles = self.filter_beads(circles[0]) #Filter the beads across the expected width (and not too small)
        # A number of circles with each circle in form = (x,y,radius)
        return circles 
    

In [10]:
class Bead(ImageStack):
    """
    Main purpose is to extract intensity with varying statistics for a single bead 
    """
    def __init__(self,bead_nd2_fpath, circle, mask_id, mono_channel=0):
        super().__init__(bead_nd2_fpath)
        self.circle = list(map(int,circle)) #circle = x,y,r
        self.mono_image = self.nd2image.get_frame(mono_channel)
        self.image_shape = self.mono_image.shape
        self.ring_thickness = int(30)
        self.mask_id = mask_id

    def get_areas(self):
        x0,y0,r0 = self.circle
        delta = self.ring_thickness/2
        outer_circle_area = np.pi * float(r0+delta) * float(r0+delta)
        inner_circle_area = np.pi * float(r0-delta) * float(r0-delta)
        ring_area = 2 * np.pi * float(r0)*self.ring_thickness
        return [outer_circle_area, ring_area, inner_circle_area]
        
    def draw_mask(self,**kwargs):
        empty_img_mask = np.zeros(self.image_shape,dtype=np.uint16)
        cv2.circle(empty_img_mask,kwargs['center'],kwargs['radius'],kwargs['color'],kwargs['thickness'])
        return empty_img_mask
     
    def get_values_under_mask(self,mask_array,idx):
        raw_image = self.nd2image.get_frame(idx)
        _image_masked_ = np.multiply(raw_image,mask_array)
        sum_bead = np.sum(_image_masked_)
        mean_bead = np.nanmean(np.where(_image_masked_!=0, _image_masked_, np.nan))
        median_bead = np.nanmedian(np.where(_image_masked_!=0, _image_masked_, np.nan))
        return sum_bead,mean_bead,median_bead
        
    def process_circle(self):
        # Initialize
        row_values = list()
        
        x0,y0,r0 = self.circle
        delta = self.ring_thickness/2
        outer_circle_params = dict(center=(x0,y0),radius=int(r0+delta),color=1,thickness=-1)
        inner_circle_params = dict(center=(x0,y0),radius=int(r0-delta),color=1,thickness=-1)
        ring_params = dict(center=(x0,y0),radius=int(r0),color=1,thickness=self.ring_thickness)
        
        # Create bead mask
        outer_circ_mask = self.draw_mask(**outer_circle_params)
        inner_circ_mask = self.draw_mask(**inner_circle_params)
        ring_mask = self.draw_mask(**ring_params)
        
        # Calculate intensity for each channel
        # Looping through the different masks modes
        for circ_mask in (outer_circ_mask,inner_circ_mask, ring_mask):
            # Looping to extract -- Sum, Mean, Median, Ratio
            for  idx,channel_name in enumerate(self.channels()):
                sum_bead, mean_bead, median_bead = self.get_values_under_mask(circ_mask,idx)
                row_values.extend([sum_bead,mean_bead,median_bead])
        return row_values

    def compile_row(self,row_values):
        complete_row_list = list()
        basic_info_row_values = [self.fname(),self.frames()[0],self.mask_id,self.circle[0],self.circle[1],self.circle[2]] 
        complete_row_list.extend(basic_info_row_values)
        
        area_row_values = self.get_areas()
        complete_row_list.extend(area_row_values)
        complete_row_list.extend(row_values)
        
        return list(complete_row_list)

In [13]:
hough_params = dict(dp=1.5,minDist=250,param1=45,param2=45,minRadius=100,maxRadius=700)
beadimage = Beads(test_nd2_fpath)
circles = beadimage.find_beads(**hough_params)
complete_row_list = list()

from tqdm import trange 

def process_beads(circles):
    print (len(circles))
    for mask_id, circle in enumerate(circles):
        bead = Bead(test_nd2_fpath,circle,mask_id)
        bead_values = bead.process_circle()
        row_list = bead.compile_row(bead_values)
        complete_row_list.append(row_list)

for i in trange(len(circles),desc="number of beads"):
    process_beads(circles)
    
beadimage.make_dataframe(complete_row_list)




number of beads:   0%|          | 0/26 [00:00<?, ?it/s]

26


number of beads:   0%|          | 0/26 [00:26<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
from tqdm.notebook import trange, tqdm
from time import sleep

for i in trange(3, desc='1st loop'):
    for j in tqdm(range(100), desc='2nd loop'):
        sleep(0.01)

In [789]:
t.metadata

{'height': 2044,
 'width': 2048,
 'date': datetime.datetime(2020, 10, 16, 21, 53, 54),
 'fields_of_view': [0],
 'frames': [0],
 'z_levels': range(0, 1),
 'z_coordinates': [2490.75],
 'total_images_per_channel': 1,
 'channels': ['Mono', 'DAPI', 'FITC', 'TRITC', 'Cy5'],
 'pixel_microns': 0.637366556218459,
 'num_frames': 1,
 'experiment': {'description': 'ND Acquisition',
  'loops': [{'start': 0,
    'duration': 0,
    'stimulation': False,
    'sampling_interval': 0.0}]},
 'events': []}

In [318]:
type(a)

dict